In [ ]:
p

In [9]:
import streamlit as st
import re
from langchain_community.tools import YouTubeSearchTool
import ast
from utilities.llms import llm_4

def youtube_streamlit(prompt):
    youtube = YouTubeSearchTool()
    result = youtube.run(prompt)
    # Convert the string representation of the list back into a Python list
    result_list = ast.literal_eval(result)

    # Extract the first link
    first_link = result_list[0]
    
    # Extract video ID from the URL
    video_id = re.search(r'(?<=v=)[^&#]+', first_link).group()

    # Construct the thumbnail URL
    thumbnail_url = f'https://img.youtube.com/vi/{video_id}/0.jpg'

    # Display the thumbnail using Streamlit
    st.image(thumbnail_url, caption='YouTube Video Thumbnail')

    # Use a markdown to display a clickable link with custom text
    display_text = "Watch Video"  # Custom text for the link
    st.markdown(f"[{display_text}]({first_link})", unsafe_allow_html=True)

    #llm_4 to descibe the video
    response = "here you link"
    return response

youtube_streamlit("show me the the song called always")

'here you link'

In [29]:
from IPython.display import Image, display
import re

# Example YouTube video URL
video_url = 'https://www.youtube.com/watch?v=37VZu3b045k'

# Extract video ID from the URL
video_id = re.search(r'(?<=v=)[^&#]+', video_url).group()

# Construct the thumbnail URL
thumbnail_url = f'https://img.youtube.com/vi/{video_id}/0.jpg'

# Display the thumbnail
display(Image(url=thumbnail_url))

In [1]:
import pandas as pd

df = pd.read_csv(r'Documents\anime.csv')
df.columns = [col.upper() for col in df.columns]
df.drop(['OTHER NAME', 'NAME','POPULARITY','FAVORITES','SCORED BY','MEMBERS'], axis='columns', inplace=True)
#df = df.rename(columns={'English name': 'Name'})


df.head()


,ANIME_ID,ENGLISH NAME,SCORE,GENRES,SYNOPSIS,TYPE,EPISODES,AIRED,PREMIERED,STATUS,PRODUCERS,LICENSORS,STUDIOS,SOURCE,DURATION,RATING,RANK,IMAGE URL
0,1,Cowboy Bebop,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",spring 1998,Finished Airing,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: The Movie,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",UNKNOWN,Finished Airing,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",spring 1998,Finished Airing,Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",summer 2002,Finished Airing,"Bandai Visual, Dentsu, Victor Entertainment, T...","Funimation, Bandai Entertainment",Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Beet the Vandel Buster,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",fall 2004,Finished Airing,"TV Tokyo, Dentsu",Illumitoon Entertainment,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,https://cdn.myanimelist.net/images/anime/7/215...


In [1]:
from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from utilities.llms import llm_4
from utilities.function import dmac_qa, youtube



tools = [
    Tool.from_function(
        name="General Chat",
        description="For general chat not covered by other tools",
        func=llm_4.invoke,
        return_direct=True
    ),
    Tool.from_function(
        name="Damac agreement",
        description="Helpful when users inquire about agreements related to Damac, alway use this tool when you can't answer spcific question about damac",
        func=dmac_qa,
        return_direct=True,
    ),
    Tool.from_function(
        name="Youtube search",
        description="For proving youtube url to user",
        func=youtube,
        return_direct=True,
    )
        
]

memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=3,
    return_messages=True,
)

agent_prompt = PromptTemplate.from_template(
    """
        You general chat assistant helper
        Be as helpful as possible and return as much information as possible.

        TOOLS:
        ------

        You have access to the following tools:

        {tools}

        To use a tool, please use the following format:

        ```
        Thought: Do I need to use a tool? Yes
        Action: the action to take, should be one of [{tool_names}]
        Action Input: the input to the action
        Observation: the result of the action
        ```
        When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

        ```
        Thought: Do I need to use a tool? No
        Final Answer: [your response here]
        ```

        Begin!

        Previous conversation history:
        {chat_history}

        New input: {input}
        {agent_scratchpad}
        
        """)

agent = create_react_agent(llm_4, tools, agent_prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

def generate_response(prompt):
    """
    Create a handler that calls the Conversational agent
    and returns a response to be rendered in the UI
    """

    response = agent_executor.invoke({"input": prompt})
    response = response['output']
    response = response.replace('\n```', '')
    return response

c:\Users\(Ai)AiSukmoren\Desktop\PERSONAL- PROJECT\chatbot\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.azure_openai.AzureOpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


In [3]:
generate_response('hi')



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
Final Answer: Hello! How can I assist you today?
```

> Finished chain.


'Hello! How can I assist you today?'

In [77]:
from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from utilities.llms import llm_4
from utilities.function import dmac_qa


def youtube_lol(prompt):
    youtube = YouTubeSearchTool()
    result = youtube.run(prompt)
    # Convert the string representation of the list back into a Python list
    result_list = ast.literal_eval(result)

    # Extract the first link
    first_link = result_list[0]
    return f'your URL link to video is {first_link}'

tools = [
    Tool.from_function(
        name="General Chat",
        description="For general chat not covered by other tools",
        func=llm_4.invoke,
        return_direct=True
    ),
    Tool.from_function(
        name="Damac agreement",
        description="Helpful when users inquire about agreements related to Damac, alway use this tool when you can't answer spcific question about damac",
        func=dmac_qa,
        return_direct=True,
    ),
    Tool.from_function(
        name="Youtube search",
        description="For proving youtube url to user",
        func=youtube_lol,
        return_direct=True,
    )
        
]

memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=3,
    return_messages=True,
)

agent_prompt = PromptTemplate.from_template(
    """
        You general chat assistant helper
        Be as helpful as possible and return as much information as possible.

        TOOLS:
        ------

        You have access to the following tools:

        {tools}

        To use a tool, please use the following format:

        ```
        Thought: Do I need to use a tool? Yes
        Action: the action to take, should be one of [{tool_names}]
        Action Input: the input to the action
        Observation: the result of the action
        ```
        When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

        ```
        Thought: Do I need to use a tool? No
        Final Answer: [your response here]
        ```

        Begin!

        Previous conversation history:
        {chat_history}

        New input: {input}
        {agent_scratchpad}
        
        """)

agent = create_react_agent(llm_4, tools, agent_prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

def generate_response(prompt):
    """
    Create a handler that calls the Conversational agent
    and returns a response to be rendered in the UI
    """

    response = agent_executor.invoke({"input": prompt})
    response = response['output']
    response = response.replace('\n```', '')
    return response


generate_response('yo')



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
Final Answer: Hey there! How can I assist you today?
```

> Finished chain.


'Hey there! How can I assist you today?'

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from utilities.llms import llm_4, embeddings
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
import streamlit as st
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQA
from langchain_community.tools import YouTubeSearchTool


def dmac_qa(question):
    """
    Use the  Vector Search Index
    to augment the response from the LLM
    """
    embedding = embeddings

    damac_vectorstore = FAISS.load_local("damac_database", embedding)

    template = """
    You are an expert female lawyer who is specialized in ansewering various ploblem regard to the agreement between 2 parties.

    If no data is returned, do not attempt to answer the question.
    Do not include any explanations or apologies in your responses. 


    Question: {question}
    context: {context}
    """

    summary_template = PromptTemplate(input_variables=['question','context'],template= template)
    
    retriever = damac_vectorstore.as_retriever()
    dmac_qa = RetrievalQA.from_llm(
        llm=llm_4, 
        retriever=retriever,
        prompt=summary_template,
        verbose=True
        )
    result = dmac_qa.run(question)
    return str(result)

def cypher_qa(prompt):
    graph = Neo4jGraph(
    url="bolt://18.207.205.88:7687",
    username="neo4j",
    password="inclinations-odors-laser", 
    )

    CYPHER_GENERATION_TEMPLATE = """
    You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
    Convert the user's question based on the schema.

    Use only the provided relationship types and properties in the schema.
    Do not use any other relationship types or properties that are not provided.

    Fine Tuning:

    For movie titles that begin with "The", move "the" to the end. For example "The 39 Steps" becomes "39 Steps, The" or "the matrix" becomes "Matrix, The".


    Schema:
    {schema}

    Question:
    {question}

    Cypher Query:
    """

    cypher_prompt = PromptTemplate.from_template(CYPHER_GENERATION_TEMPLATE)
    
    cypher_chain = GraphCypherQAChain.from_llm(
        llm=llm_4,
        graph=graph,
        cypher_prompt=cypher_prompt,
        verbose=True
    )

    result = cypher_chain.run(prompt)
    return str(result)

def kg_qa(prompt):
    """
    Use the Neo4j Vector Search Index
    to augment the response from the LLM
    """
    neo4jvector = Neo4jVector.from_existing_index(
    embeddings,  
    url=st.secrets["NEO4J_URI"],  
    username=st.secrets["NEO4J_USERNAME"],  
    password=st.secrets["NEO4J_PASSWORD"],  
    index_name="moviePlots",  
    node_label="Movie",  
    text_node_property="plot", 
    embedding_node_property="plotEmbedding",  
    retrieval_query="""
    RETURN
        node.plot AS text,
        score,
        {
            title: node.title,
            directors: [ (person)-[:DIRECTED]->(node) | person.name ],
            actors: [ (person)-[r:ACTED_IN]->(node) | [person.name, r.role] ],
            tmdbId: node.tmdbId,
            source: 'https://www.themoviedb.org/movie/'+ node.tmdbId
        } AS metadata
    """,
    )

    retriever = neo4jvector.as_retriever()

    qa = RetrievalQA.from_chain_type(
        llm_4,  
        chain_type="stuff",  
        retriever=retriever, 
    )
    # Handle the response
    result = qa.run(prompt)
    return str(result)

# youtube serch tool
youtube = YouTubeSearchTool()

In [1]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import TokenTextSplitter
from openai import AzureOpenAI
from langchain.graphs import Neo4jGraph
from langchain.graphs import Neo4jGraph
from langchain.text_splitter import TokenTextSplitter
from neo4j.exceptions import ClientError
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [151]:
# Initialize the PDF loader with the file path
loader = PyMuPDFLoader("Documents\damac.pdf")

# Load and extract data from the PDF document
data = loader.load()
data


[Document(page_content='PRIVATE AND CONFIDENTIAL \n \n \n \n1 \n \n \nLAND LEASE AGREEMENT \nTHIS LEASE AGREEMENT (the “Agreement”) is made on [*****] (“Signing Date”) at [*****] by and \nbetween: \n(1) \nSUPSUMPHAN CO., LTD., a private limited company duly organized and validly existing under \nthe laws of Thailand, with its registered office at No. 557 Rim Klong Samsen Road, Bang Kapi \nSub-district, Huai Khwang District, Bangkok, Thailand (“Lessor”); and \n(2) \n[DARIA DATA CENTRE AND CLOUD SERVICES CO., LTD],, a private limited company duly \norganized and validly existing under the laws of Thailand, with its registered office at [1788, \nSingha Complex, 30th Floor, Unit 3001-3003, 3009-3014, New Petchaburi Road, Bangkapi \nSub-district, Huai Kwang District, Bangkok] (“Lessee”), \n(The Lessor and the Lessee are hereinafter collectively referred to as the “Parties” and individually as \na “Party”.) \nBACKGROUND \nA. \nThe Lessor is the legal owner of the Land (as defined below). \nB

In [137]:
# Initialize the Text Splitter with desired chunk size and overlap
splitter = TokenTextSplitter(chunk_size=1300, chunk_overlap=30)

# Split the input text data into smaller documents
documents = splitter.split_documents(data)

# The 'documents' variable now contains a list of smaller text chunks
documents

[Document(page_content='PRIVATE AND CONFIDENTIAL \n \n \n \n1 \n \n \nLAND LEASE AGREEMENT \nTHIS LEASE AGREEMENT (the “Agreement”) is made on [*****] (“Signing Date”) at [*****] by and \nbetween: \n(1) \nSUPSUMPHAN CO., LTD., a private limited company duly organized and validly existing under \nthe laws of Thailand, with its registered office at No. 557 Rim Klong Samsen Road, Bang Kapi \nSub-district, Huai Khwang District, Bangkok, Thailand (“Lessor”); and \n(2) \n[DARIA DATA CENTRE AND CLOUD SERVICES CO., LTD],, a private limited company duly \norganized and validly existing under the laws of Thailand, with its registered office at [1788, \nSingha Complex, 30th Floor, Unit 3001-3003, 3009-3014, New Petchaburi Road, Bangkapi \nSub-district, Huai Kwang District, Bangkok] (“Lessee”), \n(The Lessor and the Lessee are hereinafter collectively referred to as the “Parties” and individually as \na “Party”.) \nBACKGROUND \nA. \nThe Lessor is the legal owner of the Land (as defined below). \nB

In [144]:
for i,content in enumerate(documents):
    if i == 16:
        print(i,content)

16 page_content=' \nPRIVATE AND CONFIDENTIAL \n \n \n \n \n17 \n \n \n2. \nOfficial Cadastral Map of the Land [Note to Draft: To be updated to reflect the actual lease \narea] \n \n \n \nThe area as indicated by the red line (totaling 7 plots): the Land \n \nThe area as indicated by the green line: the Soi Talay Bangkok Area (“Soi Talay Bangkok”) \n \n' metadata={'source': 'Documents\\damac.pdf', 'file_path': 'Documents\\damac.pdf', 'page': 16, 'total_pages': 21, 'format': 'PDF 1.7', 'title': '', 'author': 'DLA Piper Thailand', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2021', 'producer': 'Microsoft® Word 2021', 'creationDate': "D:20240115142601+07'00'", 'modDate': "D:20240115142601+07'00'", 'trapped': ''}


In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains import  LLMChain
from solutions.llm import llm


# Use OpenAI to get ES
template = """Given the table of content: {information}, 
         Give me a list of contents.
         For example:
         [SCOPE,CODES AND REGULATIONS,GENERAL REQUIREMENTS FOR WORKING PLACES,PROTECTION AGAINST HAZARDOUS MATERIALS AND OTHER HAZARDS TO HEALTH, Hazardous Materials, Personnel Protection Equipment,STRUCTURAL DESIGN,EMERGENCY ESCAPE,FIRE PROTECTION AND DETECTION,NOISE,LIGHTING,ELECTRICITY,AIR CONDITIONING & VENTILATION,LIFTING EQUIPMENT,DRAINAGE,ERGONOMICS,SAFETY SIGNS]

         Only provide the list without explanantion
     """
# Create an instance of PromptTemplate
prompt_template = PromptTemplate(input_variables=["information"], template=template)

# Create an LLMChain instance and process the TOC
chain = LLMChain(llm=llm, prompt=prompt_template)

pages='''1.  SCOPE

2.  REFERENCED STANDARDS

3.  DEFINITIONS

4.  PMI EXAMINATION

4.1  General

4.2  Methods of PMI Examination

4.3  Location, Timing and Extent of PMI Examination

4.4  Identification of Acceptable Materials

4.5  Rejected Materials

4.6  Recording and Documentation

'''
data = chain.run(information=pages)

In [6]:
print(data)

[SCOPE, REFERENCED STANDARDS, DEFINITIONS, PMI EXAMINATION, General, Methods of PMI Examination, Location, Timing and Extent of PMI Examination, Identification of Acceptable Materials, Rejected Materials, Recording and Documentation]


In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from solutions.llm import llm, embeddings

def dmac_qa(question):
    """
    Use the  Vector Search Index
    to augment the response from the LLM
    """

    damac_vectorstore = FAISS.load_local("damac_database", embeddings)

    template = """
    You are an expert female lawyer who is specialized in ansewering various ploblem regard to the agreement between 2 parties.

    If no data is returned, do not attempt to answer the question.
    Do not include any explanations or apologies in your responses. 


    Question: {question}
    context: {context}
    """

    summary_template = PromptTemplate(input_variables=['question','context'],template= template)
    
    retriever = damac_vectorstore.as_retriever()
    dmac_qa = RetrievalQA.from_llm(
        llm=llm, 
        retriever=retriever,
        prompt=summary_template,
        verbose=True
        )
    result = dmac_qa.run(question)
    return str(result)

dmac_qa("what is the agreement about")

c:\Users\(Ai)AiSukmoren\Desktop\chatbot\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


'The agreement is a land lease agreement between SUPSUMPHAN CO., LTD. (Lessor) and DARIA DATA CENTRE AND CLOUD SERVICES CO., LTD. (Lessee) for the lease of land where the Lessee wishes to establish its first data center in Thailand.'

In [1]:
class animal():
    def __init__(self,name):
        self.name = name

In [ ]:
my_animal

In [6]:
from langchain.chains import  LLMChain
from langchain import PromptTemplate
from langchain.prompts import PromptTemplate


#assign variables
input_language = 'EN'
output_language = 'TH'

#promptTemplate
summary_templates = """
   You are a translation specialist with 35 years of experience in translating from {input_language} to {output_language}. Given the question {Question}, 
   your task is to translate it into {output_language}. If the question is already in {output_language}, check for any grammatical mistakes and correct them if necessary.
    Caution:
    Provide only the answer, without any explanations.
    """

summary_prompts_template = PromptTemplate(input_variables=['Question','input_language','output_language'], template = summary_templates)

chain = LLMChain(llm=llm, prompt=summary_prompts_template) 

def translate_qa(prompt):
    Answer = chain.run(Question=prompt,input_language=input_language,output_language=output_language)
    print(str(Answer))  
    
translate_qa("how are you today")

คุณสบายดีไหมวันนี้


In [5]:
from langchain_community.tools import YouTubeSearchTool

# youtube serch tool
youtube = YouTubeSearchTool()

youtube.invoke("provide me the song name ")

ModuleNotFoundError: No module named 'youtube_search'

In [8]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from solutions.llm import llm


def movie_qa(prompt):
    graph = Neo4jGraph(
    url="bolt://18.207.205.88:7687",
    username="neo4j",
    password="inclinations-odors-laser", 
    )

    CYPHER_GENERATION_TEMPLATE = """
    You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
    Convert the user's question based on the schema.

    Use only the provided relationship types and properties in the schema.
    Do not use any other relationship types or properties that are not provided.

    Fine Tuning:

    For movie titles that begin with "The", move "the" to the end. For example "The 39 Steps" becomes "39 Steps, The" or "the matrix" becomes "Matrix, The".


    Schema:
    {schema}

    Question:
    {question}

    Cypher Query:
    """

    cypher_prompt = PromptTemplate.from_template(CYPHER_GENERATION_TEMPLATE)
    
    cypher_chain = GraphCypherQAChain.from_llm(
        llm=llm,
        graph=graph,
        cypher_prompt=cypher_prompt,
        verbose=True
    )

    result = cypher_chain.invoke({prompt})
    return result['result']
    
    
movie_qa("who acted in toy story")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)
WHERE m.title = 'Toy Story'
RETURN a.name
Full Context:
[{'a.name': 'Jim Varney'}, {'a.name': 'Tim Allen'}, {'a.name': 'Tom Hanks'}, {'a.name': 'Don Rickles'}]

> Finished chain.


'Jim Varney, Tim Allen, Tom Hanks, and Don Rickles acted in Toy Story.'

In [9]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from solutions.llm import llm

def movie_qa(prompt):
    graph = Neo4jGraph(
    url="bolt://18.207.205.88:7687",
    username="neo4j",
    password="inclinations-odors-laser", 
    )

    CYPHER_GENERATION_TEMPLATE = """
    You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
    Convert the user's question based on the schema.

    Instructions:
    Use only the provided relationship types and properties in the schema.
    Do not use any other relationship types or properties that are not provided.
    For movie titles that begin with "The", move "the" to the end, For example "The 39 Steps" becomes "39 Steps, The" or "The Matrix" becomes "Matrix, The".

    If no data is returned, do not attempt to answer the question.
    Only respond to questions that require you to construct a Cypher statement.
    Do not include any explanations or apologies in your responses.

    Examples:

    Find movies and genres:
    MATCH (m:Movie)-[:IN_GENRE]->(g)
    RETURN m.title, g.name

    Schema: {schema}
    Question: {question}
    """

    cypher_generation_prompt = PromptTemplate(
        template=CYPHER_GENERATION_TEMPLATE,
        input_variables=["schema", "question"],
    )
    
    cypher_chain = GraphCypherQAChain.from_llm(
        llm=llm,
        graph=graph,
        cypher_prompt=cypher_generation_prompt,
        verbose=True
    )

    result = cypher_chain.invoke({prompt})
    return result['result']

movie_qa("toy story 2")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie) 
WHERE m.title = 'Toy Story 2' 
RETURN m
Full Context:
[{'m': {'languages': ['English', ' Arabic', ' Spanish'], 'year': 1999, 'imdbId': '0120363', 'runtime': 92, 'imdbRating': 7.9, 'movieId': '3114', 'countries': ['USA'], 'imdbVotes': 369499, 'title': 'Toy Story 2', 'url': 'https://themoviedb.org/movie/863', 'revenue': 497366869, 'tmdbId': '863', 'plot': 'When Woody is stolen by a toy collector, Buzz and his friends vow to rescue him, but Woody finds the idea of immortality in a museum tempting.', 'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/yFWQkz2ynjwsazT6xQiIXEUsyuh.jpg', 'released': '1999-11-24', 'budget': 90000000}}]

> Finished chain.


'"Toy Story 2" is a movie that was released in 1999. The plot revolves around Woody being stolen by a toy collector, leading Buzz and his friends to vow to rescue him. However, Woody finds the idea of immortality in a museum tempting. The movie is in English, Arabic, and Spanish and was produced in the USA. It has a runtime of 92 minutes and an IMDB rating of 7.9 based on 369,499 votes. The movie had a budget of $90,000,000 and generated a revenue of $497,366,869. You can find more information about the movie on this [link](https://themoviedb.org/movie/863).'

In [3]:
from langchain.chains import  LLMChain
from langchain import PromptTemplate
from langchain.prompts import PromptTemplate
from solutions.llm import llm


#assign variables
input_language = 'EN'
output_language = 'TH'

#promptTemplate
summary_templates = """
   You are a translation specialist with 35 years of experience in translating from {input_language} to {output_language}. Given the question {Question}, 
   your task is to translate it into {output_language}. If the question is already in {output_language}, check for any grammatical mistakes and correct them if necessary.
    Caution:
    Provide only the answer, without any explanations.
    """

summary_prompts_template = PromptTemplate(input_variables=['Question','input_language','output_language'], template = summary_templates)

chain = LLMChain(llm=llm, prompt=summary_prompts_template) 

def translate_qa(prompt):
    Answer = chain.run(Question=prompt,input_language=input_language,output_language=output_language)
    print(str(Answer))  
    
translate_qa("how are you today")

คุณเป็นอย่างไรบ้างวันนี้


In [5]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from solutions.llm import llm

graph = Neo4jGraph(
    url="bolt://18.207.205.88:7687",
    username="neo4j",
    password="inclinations-odors-laser", 
)

CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
For movie titles that begin with "The", move "the" to the end, For example "The 39 Steps" becomes "39 Steps, The" or "The Matrix" becomes "Matrix, The".

If no data is returned, do not attempt to answer the question.
Only respond to questions that require you to construct a Cypher statement.
Do not include any explanations or apologies in your responses.

Examples:

Find movies and genres:
MATCH (m:Movie)-[:IN_GENRE]->(g)
RETURN m.title, g.name

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)


def movie_qa(prompt):
    cypher_chain = GraphCypherQAChain.from_llm(
        llm=llm,
        graph=graph,
        cypher_prompt=cypher_generation_prompt,
        verbose=True
    )

    result = cypher_chain.run(prompt)
    print(str(result))

movie_qa("who is the actor of toy story")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Toy Story"})-[:ACTED_IN]-(a:Actor)
RETURN a.name
Full Context:
[{'a.name': 'Jim Varney'}, {'a.name': 'Tim Allen'}, {'a.name': 'Tom Hanks'}, {'a.name': 'Don Rickles'}]

> Finished chain.
The actors of Toy Story are Jim Varney, Tim Allen, Tom Hanks, and Don Rickles.


In [8]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from solutions.llm import llm


def dmac_qa(prompt):
    """
    Use the  Vector Search Index
    to augment the response from the LLM
    """
    embeddings = OpenAIEmbeddings()
    damac_vectorstore = FAISS.load_local("damac_database", embeddings)
    retriever = damac_vectorstore.as_retriever()
    dmac_qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    verbose=True
    )
    result = dmac_qa.run(prompt)
    return str(result)

prompt = "what is the lease amount"
dmac_qa(prompt)

c:\Users\(Ai)AiSukmoren\.conda\envs\langchain_demo\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


'The lease amount is 17,000,000 Thailand Bath (seventeen million THB) per year.'

In [9]:
from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from solutions.llm import llm
from solutions.damac import dmac_qa
from solutions.youtube import youtube
from solutions.graph import movie_qa
from solutions.translator import translate_qa
from langchain.prompts import PromptTemplate



tools = [
    Tool.from_function(
        name="General Chat",
        description="For general chat not covered by other tools",
        func=llm.invoke,
        return_direct=True,
    ),
    Tool.from_function(
        name="Damac agreement",
        description="Provides information about DAMAC agreement",
        func=dmac_qa,
        return_direct=True,
    ),
    Tool.from_function(
        name="Youtube search",
        description="For proving youtube url to user",
        func=youtube.run,
        return_direct=True,
    ),
        Tool.from_function(
        name="Movie serch",
        description="For proving details and information related to movie",
        func=movie_qa,
        return_direct=True,
    ),
        Tool.from_function(
        name="Translator",
        description="Use for when user want to translate the result to TH",
        func=translate_qa,
        return_direct=True,
    ),
        
]

memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=3,
    return_messages=True,
)

agent_prompt = PromptTemplate.from_template(
    """
        You are a lawyer who is specilized in answering ansewer regard to law and regulation decision question.
        Be as helpful as possible and return as much information as possible.

        TOOLS:
        ------

        You have access to the following tools:

        {tools}

        To use a tool, please use the following format:

        ```
        Thought: Do I need to use a tool? Yes
        Action: the action to take, should be one of [{tool_names}]
        Action Input: the {input} to the action
        Observation: the result of the action
        ```

        When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

        ```
        Thought: Do I need to use a tool? No
        Final Answer: [your response here]
        ```

        Begin!

        Previous conversation history:
        {chat_history}

        New input: {input}
        {agent_scratchpad}
        """
)
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)




def generate_response(prompt):
    """
    Create a handler that calls the Conversational agent
    and returns a response to be rendered in the UI
    """

    response = agent_executor.invoke({"input": prompt})

    return response['output']

while True:
    q = input("> ")
    if q.lower() == "exit":
        print("Exiting...")
        break
    else:
        result = generate_response(q)


        print(result)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Movie serch
Action Input: E.TI'm sorry, but I don't know the answer.


> Finished chain.
I'm sorry, but I don't know the answer.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Movie serch
Action Input: who acted in toy storyJim Varney, Tim Allen, Tom Hanks, and Don Rickles acted in Toy Story.


> Finished chain.
Jim Varney, Tim Allen, Tom Hanks, and Don Rickles acted in Toy Story.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Hello! How can I assist you today?

> Finished chain.
Hello! How can I assist you today?


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Toy Story 2 was directed by John Lasseter, co-directed by Ash Brannon and Lee Unkrich.

> Finished chain.
Toy Story 2 was directed by John Lasseter, co-directed by Ash Brannon and Lee Unkrich.


> Entering new AgentExecu

In [9]:
import streamlit as st
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQA
from solutions.llm import llm, embeddings

def kg_qa(prompt):
    """
    Use the Neo4j Vector Search Index
    to augment the response from the LLM
    """
    neo4jvector = Neo4jVector.from_existing_index(
    embeddings,  
    url=st.secrets["NEO4J_URI"],  
    username=st.secrets["NEO4J_USERNAME"],  
    password=st.secrets["NEO4J_PASSWORD"],  
    index_name="moviePlots",  
    node_label="Movie",  
    text_node_property="plot", 
    embedding_node_property="plotEmbedding",  
    retrieval_query="""
    RETURN
        node.plot AS text,
        score,
        {
            title: node.title,
            directors: [ (person)-[:DIRECTED]->(node) | person.name ],
            actors: [ (person)-[r:ACTED_IN]->(node) | [person.name, r.role] ],
            tmdbId: node.tmdbId,
            source: 'https://www.themoviedb.org/movie/'+ node.tmdbId
        } AS metadata
    """,
    )

    retriever = neo4jvector.as_retriever()

    kg_qa = RetrievalQA.from_chain_type(
        llm,  
        chain_type="stuff",  
        retriever=retriever,  
        verbose=True
    )
    # Handle the response
    result = kg_qa.run(prompt)
    return str(result)

kg_qa("what is the plot about jumanji")



> Entering new RetrievalQA chain...

> Finished chain.


'The plot of Jumanji revolves around two kids who find and play a magical board game. By doing so, they inadvertently release a man who has been trapped inside the game for decades. As they continue to play, they encounter various dangers and challenges that can only be stopped by finishing the game. The kids must work together to navigate through the jungle-themed world of Jumanji and find a way to restore order and return everything back to normal.'